In [19]:
import random
import math
import numpy as np

In [8]:
class Data:

    # initialize needed values
    def __init__(self, data_address):
        self.address    = data_address      # address of test file
        self.degree     = 0                 # degree of polynomial
        self.low        = 0                 # lower bound of coefs
        self.high       = 0                 # upper bound of coefs
        self.num_points = 0                 # number of points in test case
        self.points     = []                # points of curve

    # read test case
    def load_data(self):
        with open (self.address, 'r') as f:
            self.degree         = int(f.readline())     
            self.low, self.high = map(float, f.readline().split(" "))  
            self.num_points     = int(f.readline())  
            self.points         = [tuple(map(float, f.readline().split(" "))) for _ in range(self.num_points)]  # append the point to list of points  


In [102]:
class Gene:

    # initialize parameters
    def __init__(self, degree, points, low, high):
        self.gene = []
        self.degree = degree
        self.points = points
        self.low = low
        self.high = high
        self.fitness = 0


    # create random gene for first population
    def initial_gene(self):
        self.gene = [random.uniform(self.low, self.high) for _ in range(self.degree)]


    # calculate fitness 
    def fitness_function(self):
        # SRR = sigma_{i=0}{n} (pred-y)^2
        self.fitness = sum((np.polyval(self.gene, point[0]) - point[1])**2 for point in self.points)
        return self.fitness
    
    # mutate gene
    def mutation(self):
        i = random.randint(0, self.degree-1)

        # randomely change one of the coefs in gene
        self.gene[i] = random.uniform(self.low, self.high)
        self.fitness_function()



In [87]:
class Population:

    # initialize population's parameters
    def __init__(self, data, pop_size=100):
        self.population_size             = pop_size
        self.population                  = []
        self.points                  = data.points
        self.low, self.high, self.degree = data.low, data.high, data.degree

    # generate first population randomly
    def initialize_population(self):
        for _ in range(self.population_size):
            genei = Gene(self.degree, self.points, self.low, self.high)
            genei.initial_gene()
            self.population.append(genei)
    

    # selest parents from population for crossover using tornoment selection
    def parent_selection(self, N=3):
        # choose random N candidates to attend tornoment
        father_candidates = [random.choice(self.population) for _ in range(N)]
        mother_candidates = [random.choice(self.population) for _ in range(N)]

        # choose the best one of N candidates to be a parent
        # find candidates fitnesses
        father_vals = [(father.fitness_function(), father) for father in father_candidates]
        mother_vals = [(mother.fitness_function(), mother) for mother in mother_candidates]
        # sort candidates
        sorted_fathers = sorted(father_vals, key=lambda x: x[0])
        sorted_mothers = sorted(mother_vals, key=lambda x: x[0])
        # select parents with min fitness
        best_father = sorted_fathers[0][1]
        best_mother = sorted_mothers[0][1]
        
        return best_father, best_mother


    # crossover function
    def crossover(self, father, mother):

        child1l, child2l = [], []
        child1, child2 = Gene(self.degree, self.points, self.low, self.high), Gene(self.degree, self.points, self.low, self.high)
        random_point = random.randint(0, self.degree-1)    # select a random point and cut 1 of parents for each child from there

        # put first part of 1 parent to each child
        child1l = [father.gene[i] for i in range(random_point)]
        child2l = [mother.gene[i] for i in range(random_point)]

        # put the rest of other parent
        for i in range(random_point, self.degree):
            child1l.append(mother.gene[i])
            child2l.append(father.gene[i])

        # initialize childs gene object
        child1.gene = child1l
        child2.gene = child2l
        child1.fitness_function()
        child2.fitness_function()
        
        return child1, child2



In [104]:
# load test case
curve_test1 = Data("CurveFitting-Tests/CurveFitting_test1.txt")
curve_test1.load_data()

# create first population
population = Population(curve_test1, pop_size=100)
population.initialize_population()

# select 2 parents for crossover
father, mother = population.parent_selection(N=4)

# crossover 2 selected parents with p_cross
p_cross = 0.75
e_cross = random.random()
if e_cross < p_cross:
    child1, child2 = population.crossover(father, mother)
    
# mutate childs with p_mutate
p_mutate = 0.5
e_mutate1 = random.random()
if e_mutate1 < p_mutate:
    child1.mutation()
e_mutate2 = random.random()
if e_mutate2 < p_mutate:
    child2.mutation()